# Preparation for Lesson 5: Data management


Our example is inspired by the Dutch LISS data, in particular the waves on Time Use and consumption in November 2019 and in April 2020. 

In particular, the data contain the following variables, alphabetically sorted:

| Variable     | Content                                                           |
|:-------------|:------------------------------------------------------------      |
| geslacht     | Gender (Man: Male, Vrouw: Female)                                 |
| nohouse_encr | Household identifier                                              |
| nomem_encr   | Member identifier                                                 |
| v1q1_v1col1  | Working hours (Nov) / Working hours at workplace (Apr)            |
| v1q1a_v1col1 | Working hours in home office, no kids in HH (Apr)                 |
| v1q1b_v1col1 | Working hours in home office while responsible for kids (Apr)     |
| v1q1c_v1col1 | Working hours in home office while not responsible for kids (Apr) |
| v1q5_v1col1  | Childcare hours (all in Nov, residual in Apr)                     |
| v1q5a_v1col1 | Homeschooling hours (Apr)                                         |


In this exercise, we will only work with the November data.

We start by importing Pandas

In [1]:
import pandas as pd

## Read in the data

The November data is stored in Stata format. Replace the `XXXXX` in the next cell by the appropriate Pandas function.

In [58]:
nov_2019 = pd.read_stata("time_use_2019-11.dta", convert_categoricals=True)
nov_2019_1 = pd.read_stata("time_use_2019-11.dta", convert_categoricals=False)

## Browse the data

You can browse through the data by just typing the name of a DataFrame as the last thing in a cell.

This will yield a nice html-formatted output. Use this to find out the difference between setting `convert_categoricals` to `True` or `False` in the call above. In case you know some Stata: Can you explain what is happening? Else don't worry about the reasons behind what is going on.


In [59]:
print(nov_2019)
print(nov_2019_1)

    nomem_encr geslacht  v1q1_v1col1  v1q5_v1col1  nohouse_encr
0    1687033.0      Man          0.0          2.0     1049420.0
1    1662353.0    Vrouw          9.0          0.0     1049420.0
2    1631191.0      Man         67.0          1.0     1011033.0
3    1687630.0    Vrouw         17.0          6.0     1011033.0
4    1746405.0      Man         40.0         12.0     1047651.0
..         ...      ...          ...          ...           ...
83   1700319.0      Man         40.0          NaN     1132053.0
84   1696174.0      Man          0.0          NaN     1156059.0
85   1678816.0    Vrouw         24.0         25.0     1144468.0
86   1668177.0      Man         36.0          4.0     1144468.0
87   1655142.0      Man          0.0          NaN     1159704.0

[88 rows x 5 columns]
    nomem_encr  geslacht  v1q1_v1col1  v1q5_v1col1  nohouse_encr
0    1687033.0         0          0.0          2.0     1049420.0
1    1662353.0         1          9.0          0.0     1049420.0
2    1631191.0

## Rename variables

We give the Dutch and partly cryptic variable names sensible identifiers. The `replace` method on DataFrames returns a new DataFrame, so we need to assign it to `nov_2019` again if we want to continue working with it.

Note that this is a very stateful transformation if you assign to the same variable; you will not be able to successfully execute the cell twice without re-loading the data above.

In [60]:
nov_2019 = nov_2019.rename(
    columns={
        "geslacht": "gender",
        "v1q1_v1col1": "working_hours",
        "v1q5_v1col1": "childcare_hours"
    }
)
nov_2019

,nomem_encr,gender,working_hours,childcare_hours,nohouse_encr
0,1687033.0,Man,0.0,2.0,1049420.0
1,1662353.0,Vrouw,9.0,0.0,1049420.0
2,1631191.0,Man,67.0,1.0,1011033.0
3,1687630.0,Vrouw,17.0,6.0,1011033.0
4,1746405.0,Man,40.0,12.0,1047651.0
...,...,...,...,...,...
83,1700319.0,Man,40.0,NaN,1132053.0
84,1696174.0,Man,0.0,NaN,1156059.0
85,1678816.0,Vrouw,24.0,25.0,1144468.0
86,1668177.0,Man,36.0,4.0,1144468.0


## Convert data types

`nomem_encr` and `nohouse_encr` are classical identifiers. They can be used to identify a particular observation, but they do not carry any meaning beyond that.

You have heard in the screencast that you should never use floating point numbers for identifiers. It is common that this happens in Stata, though (e.g., through use of `compress` or mathematical operations of integers, which do implicit type conversions unless you request an integer back).

Add new columns `hh_id` and `ind_id` with sensible data types. 


In [61]:
nov_2019["hh_id"] = nov_2019.nohouse_encr.astype(int)
nov_2019["ind_id"] = nov_2019.nomem_encr.astype(int)
nov_2019

,nomem_encr,gender,working_hours,childcare_hours,nohouse_encr,hh_id,ind_id
0,1687033.0,Man,0.0,2.0,1049420.0,1049420,1687033
1,1662353.0,Vrouw,9.0,0.0,1049420.0,1049420,1662353
2,1631191.0,Man,67.0,1.0,1011033.0,1011033,1631191
3,1687630.0,Vrouw,17.0,6.0,1011033.0,1011033,1687630
4,1746405.0,Man,40.0,12.0,1047651.0,1047651,1746405
...,...,...,...,...,...,...,...
83,1700319.0,Man,40.0,NaN,1132053.0,1132053,1700319
84,1696174.0,Man,0.0,NaN,1156059.0,1156059,1696174
85,1678816.0,Vrouw,24.0,25.0,1144468.0,1144468,1678816
86,1668177.0,Man,36.0,4.0,1144468.0,1144468,1668177


## Select columns

We do not need to keep the old identifiers anymore.

You can select a subset of columns by including a list of column names in the standard square brackets used for indexing in Python.

Replace the XXXX and YYYY appropriately and include all other variables that you want to keep.

In [67]:
nov_20190 = nov_2019[["gender", "working_hours"]]
nov_20190

,gender,working_hours
0,Man,0.0
1,Vrouw,9.0
2,Man,67.0
3,Vrouw,17.0
4,Man,40.0
...,...,...
83,Man,40.0
84,Man,0.0
85,Vrouw,24.0
86,Man,36.0


## Set index

The default index created by Pandas (a DataFrame always has an index) does not make much sense.

Create an index based on a column / several columns that makes sense to you via replacing the XXXX by an appropriate construct.

In [63]:
nov_2019_with_index = nov_2019.set_index("gender")
nov_2019_with_index

,nomem_encr,working_hours,childcare_hours,nohouse_encr,hh_id,ind_id
gender,,,,,,
Man,1687033.0,0.0,2.0,1049420.0,1049420,1687033
Vrouw,1662353.0,9.0,0.0,1049420.0,1049420,1662353
Man,1631191.0,67.0,1.0,1011033.0,1011033,1631191
Vrouw,1687630.0,17.0,6.0,1011033.0,1011033,1687630
Man,1746405.0,40.0,12.0,1047651.0,1047651,1746405
...,...,...,...,...,...,...
Man,1700319.0,40.0,NaN,1132053.0,1132053,1700319
Man,1696174.0,0.0,NaN,1156059.0,1156059,1696174
Vrouw,1678816.0,24.0,25.0,1144468.0,1144468,1678816


## Our first reduction operation

Calculate the mean hours spent on different activities.

In [64]:
from statistics import mean

nov_2019["working_hours"].mean()


17.988636363636363

## Our second reduction operation

Calculate the mean hours spent on different activities by gender by appending the following with the appropriate method to calculate a mean.

In [65]:
nov_2019


,nomem_encr,gender,working_hours,childcare_hours,nohouse_encr,hh_id,ind_id
0,1687033.0,Man,0.0,2.0,1049420.0,1049420,1687033
1,1662353.0,Vrouw,9.0,0.0,1049420.0,1049420,1662353
2,1631191.0,Man,67.0,1.0,1011033.0,1011033,1631191
3,1687630.0,Vrouw,17.0,6.0,1011033.0,1011033,1687630
4,1746405.0,Man,40.0,12.0,1047651.0,1047651,1746405
...,...,...,...,...,...,...,...
83,1700319.0,Man,40.0,NaN,1132053.0,1132053,1700319
84,1696174.0,Man,0.0,NaN,1156059.0,1156059,1696174
85,1678816.0,Vrouw,24.0,25.0,1144468.0,1144468,1678816
86,1668177.0,Man,36.0,4.0,1144468.0,1144468,1668177


In [68]:
nov_2019.groupby("gender")[["working_hours","childcare_hours"]].mean()

,working_hours,childcare_hours
gender,,
Man,23.434783,7.315789
Vrouw,12.023810,18.263158
